# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [136]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway
from scipy.stats import t
from scipy.stats import sem



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [133]:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [6]:
salaries.head()



,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [36]:
salaries.isna().sum()
# this will give us the number of missing values per column


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [20]:
condition1 = salaries['Salary or Hourly'] == 'Salary'
salaries_info = salaries[condition1]['Salary or Hourly'].count()

condition2 = salaries['Salary or Hourly'] == 'Hourly'
hourly_info = salaries[condition2]['Salary or Hourly'].count()


print('There are', salaries_info, 'receiving salaried')
print('There are', hourly_info, 'receivig hourly')
print('There are more', salaries_info - hourly_info, 'workers receiving salaried')

There are 25161 receiving salaried
There are 8022 receivig hourly
There are more 17139 workers receiving salaried


In [118]:
# could also do this!
salaries['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [35]:
different_departments = list(set(salaries['Department']))

list_counts = []
for department in different_departments:
    condition = salaries['Department'] == department
    list_counts.append(salaries[condition]['Department'].count())

list_count_depart = list(zip(different_departments, list_counts))    

list_count_depart


[('FAMILY & SUPPORT', 615),
 ('ADMIN HEARNG', 39),
 ('POLICE BOARD', 2),
 ('BOARD OF ELECTION', 107),
 ('BUILDINGS', 269),
 ('BOARD OF ETHICS', 8),
 ('WATER MGMNT', 1879),
 ('COMMUNITY DEVELOPMENT', 207),
 ('OEMC', 2102),
 ('AVIATION', 1629),
 ("MAYOR'S OFFICE", 85),
 ('ANIMAL CONTRL', 81),
 ('FIRE', 4641),
 ('HEALTH', 488),
 ('GENERAL SERVICES', 980),
 ('CITY COUNCIL', 411),
 ('HUMAN RESOURCES', 79),
 ('CULTURAL AFFAIRS', 65),
 ('TREASURER', 22),
 ('HUMAN RELATIONS', 16),
 ('CITY CLERK', 84),
 ('BUDGET & MGMT', 46),
 ('INSPECTOR GEN', 87),
 ('STREETS & SAN', 2198),
 ('POLICE', 13414),
 ('LAW', 407),
 ('PROCUREMENT', 92),
 ('DISABILITIES', 28),
 ('LICENSE APPL COMM', 1),
 ('BUSINESS AFFAIRS', 171),
 ('TRANSPORTN', 1140),
 ('PUBLIC LIBRARY', 1015),
 ('DoIT', 99),
 ('COPA', 116),
 ('FINANCE', 560)]

In [119]:
# we could also do this!
salaries['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [121]:
# H0: hourly wage of all hourly workers is significantly different from $30/hr

sample = salaries['Hourly Rate'].dropna()

alpha = 0.05

test = ttest_1samp(sample, 30)

p_value = test[1]

alpha > p_value

# because alpha e higher than the p_value we reject the null hypothesis


True

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [124]:
# H0: salaries this year are higher than last year's

police_salaries = salaries.loc[salaries['Department'] == 'POLICE']

sample_s = police_salaries['Annual Salary'].dropna()

alpha = 0.05

test = ttest_1samp(sample_s, 86000)

p_value = test[1]

alpha > p_value/2


# because alpha e higher than the p_value we reject the null hypothesis



True

Using the `crosstab` function, find the department that has the most hourly workers. 

In [96]:
pd.crosstab(salaries['Hourly Rate'], salaries['Department']).sum().sort_values(ascending = False).to_frame().reset_index().iloc[0]



Department    STREETS & SAN
0                      1862
Name: 0, dtype: object

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [129]:
# H0: workers from the department with the most hourly workers have a wage lower or equal than 35

top_salaries = salaries.loc[salaries['Department'] == 'STREETS & SAN']

sample_s = top_salaries['Hourly Rate'].dropna()

alpha = 0.05

test = ttest_1samp(sample_s, 35)

p_value = test[1]

alpha > p_value/2


# because alpha e higher than the p_value we reject the null hypothesis



True

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [134]:
mean = salaries['Hourly Rate'].dropna().mean()

rows = salaries['Hourly Rate'].dropna().count()

error = sem(salaries['Hourly Rate'].dropna())

confidence_interval = t.interval(0.95, loc = mean, scale = error, df = rows - 1)

confidence_interval

(32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [135]:
mean = salaries['Annual Salary'].dropna().mean()

rows = salaries['Annual Salary'].dropna().count()

error = sem(salaries['Annual Salary'].dropna())

confidence_interval = t.interval(0.95, loc = mean, scale = error, df = rows - 1)

confidence_interval

(86526.99656774197, 87047.0030125611)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

